## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/111-googleq-a-train-bert-base-no-dropout-v2/model.h5'
BERT_PATH = '../input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
VOCAB_PATH = BERT_PATH +'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(pooled_output)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.924591,0.650844,0.211431,0.543601,0.497603,0.582725,0.575239,0.553629,0.377502,...,0.889422,0.950800,0.505522,0.978684,0.965135,0.852533,0.038060,0.031977,0.819858,0.941890
1,46,0.900351,0.530713,0.004439,0.779558,0.848290,0.935872,0.567409,0.442782,0.075652,...,0.737607,0.939594,0.648438,0.975578,0.978675,0.873913,0.973908,0.099038,0.026448,0.894503
2,70,0.941139,0.671289,0.014053,0.824923,0.882100,0.935841,0.574457,0.515430,0.140182,...,0.901620,0.884821,0.579857,0.976697,0.932654,0.798039,0.078477,0.046316,0.821042,0.941505
3,132,0.893803,0.524494,0.005759,0.731849,0.780733,0.925929,0.570924,0.462349,0.050523,...,0.745197,0.925985,0.657689,0.974029,0.980663,0.868422,0.853055,0.171065,0.491864,0.922533
4,200,0.936279,0.487041,0.021243,0.831644,0.786726,0.848384,0.555375,0.583405,0.096050,...,0.766635,0.909010,0.541389,0.979795,0.959310,0.795037,0.108389,0.072959,0.479265,0.924353


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.896759,0.599073,0.029943,0.754890,0.830815,0.873047,0.562916,0.461686,0.184505,...,0.811185,0.919654,0.633795,0.970049,0.963329,0.854658,0.544116,0.119776,0.426066,0.912083
std,2812.670060,0.047096,0.125408,0.055487,0.100725,0.090327,0.125179,0.048799,0.084107,0.170775,...,0.085221,0.025957,0.046838,0.009434,0.016618,0.037559,0.358485,0.060700,0.294205,0.026690
min,39.000000,0.699402,0.308098,0.002570,0.162795,0.439362,0.233182,0.419590,0.306254,0.016525,...,0.599724,0.798567,0.432560,0.915529,0.902728,0.706443,0.007175,0.014385,0.013193,0.824364
25%,2572.000000,0.869122,0.494005,0.005342,0.706060,0.794700,0.852143,0.529676,0.404837,0.061002,...,0.742062,0.906358,0.606450,0.965672,0.956094,0.830631,0.138165,0.078070,0.151621,0.895610
50%,5093.000000,0.902607,0.590740,0.009237,0.762996,0.839898,0.922771,0.557497,0.442344,0.116760,...,0.812457,0.924298,0.638677,0.971954,0.967572,0.861587,0.661021,0.115608,0.408081,0.915881
75%,7482.000000,0.933653,0.694491,0.023918,0.823610,0.888487,0.947228,0.591569,0.500573,0.249869,...,0.887787,0.939234,0.665826,0.976406,0.975254,0.880368,0.888982,0.156913,0.685574,0.931045
max,9640.000000,0.974734,0.872067,0.438386,0.962952,0.981970,0.980577,0.766068,0.755208,0.839418,...,0.957903,0.963753,0.741859,0.986135,0.988176,0.940129,0.977071,0.369199,0.979603,0.961918
